<big><b> Capstone Project: The Battle of Neighborhoods </b> </big>

Opening an Indian Restaurant in Houston, TX
Build a dataframe of neighborhoods in Houston, TX a by web scraping the data from Wikipedia page <br />
Get the geographical coordinates of the neighborhoods <br />
Obtain the venue data for the neighborhoods from Foursquare API <br />
Explore and cluster the neighborhoods <br />
Select the best cluster to open a new Indian Restaurant

<b> Import Libraries </b>

Import Pandas & NumPy. Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language. NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge folium=0.5.0 
!conda install -c conda-forge geopy

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following NEW packages will be INSTALLED:

    ld_impl_linux-64: 2.33.1-h53a641e_8       conda-forge               

The following packages will be UPDATED:

    ca-certificates:  2018.11.29-ha4d7672_0   conda-forge/label/cf201901 --> 2019.11.28-hecc5488_0   conda-forge
    certifi:          2018.11.29-py36_1000    conda-forge/label/cf201901 --> 2019.11.28-py36_0       conda-forge
    cryptography:     2.3.1-py36hdffb7b8_0    conda-forge/label/cf201901 --> 2.8-py36h72c5cf5_1      conda-forge
    curl:             7.63.0-h74213dd_0       conda-forge/label/cf201901 --> 7.68.0-hf8cf82a_0       conda-forge
    freetds:          1.00.97-hc365091_0                                 --> 1.1rc3-h4fe99da_0       conda-forge
    grpcio:           1.16.0-py36hd60e7a3_0   conda-forge/label/cf201901 --> 1.23.0-py36he9ae1f9_0   conda-forge
    krb5:       

<big> <b> Download and Explore Dataset </big> </b>

<b> Scrap data from Wikipedia page into a DataFrame </b>

In [3]:
!conda install -c conda-forge/label/gcc7 geocoder
!conda install -c conda-forge/label/cf201901 geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_0          52 KB  conda-forge/label/gcc7

The following packages will be UPDATED:

    geocoder:        1.38.1-py_0             conda-forge/label/cf201901 --> 1.38.1-py_0               conda-forge/label/gcc7
    libarchive:      3.3.3-h5d8350f_5                                   --> 3.3.3-ha149a29_1000       conda-forge/label/gcc7
    thrift-cpp:      0.11.0-h02b749d_3                                  --> 0.11.0-h23e226f_1003      conda-forge/label/gcc7

The following packages will be DOWNGRADED:

    ca-certificates: 2019.11.28-hecc5488_0   conda-forge                --> 2018.10.15-ha4d7672_0     conda-forge/label/gcc7
    certifi:         2019.11.28-py36_0  

In [4]:
!pip install wikipedia

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Houston").text

In [7]:
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
# create a new DataFrame from the list
houston_df = pd.DataFrame({"Neighborhood": neighborhoodList})

houston_df.head()

,Neighborhood
0,► Gated communities in Houston‎ (5 P)
1,"► Alief, Houston‎ (1 C, 1 P)"
2,"► Downtown Houston‎ (1 C, 44 P)"
3,"► East End, Houston‎ (13 P)"
4,"► Fourth Ward, Houston‎ (7 P)"


In [12]:
# print the number of rows of the dataframe
houston_df.shape

(10, 1)

<big> <b> Get the geographical coordinates </b> </big>

In [13]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Huston, Texas'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in houston_df["Neighborhood"].tolist() ]

In [15]:
coords

[[29.80698211297986, -95.28319849223183],
 [29.714976416574544, -95.58978525746251],
 [29.753970317331845, -95.3525106646128],
 [29.694615665601162, -95.2787934996728],
 [29.75762000000003, -95.38448999999997],
 [29.79713000000004, -95.39766999999995],
 [29.741460000000075, -95.37670999999995],
 [29.742718787183733, -95.32818832951054],
 [29.8035174965043, -95.51589580701445],
 [29.66358307567675, -95.22910994584586]]

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [18]:
# merge the coordinates into the original dataframe
houston_df['Latitude'] = df_coords['Latitude']
houston_df['Longitude'] = df_coords['Longitude']

In [19]:
# check the neighborhoods and the coordinates
print(houston_df.shape)
houston_df

(10, 3)


,Neighborhood,Latitude,Longitude
0,► Gated communities in Houston‎ (5 P),29.806982,-95.283198
1,"► Alief, Houston‎ (1 C, 1 P)",29.714976,-95.589785
2,"► Downtown Houston‎ (1 C, 44 P)",29.753970,-95.352511
3,"► East End, Houston‎ (13 P)",29.694616,-95.278793
4,"► Fourth Ward, Houston‎ (7 P)",29.757620,-95.384490
5,► Houston Heights‎ (10 P),29.797130,-95.397670
6,"► Midtown, Houston‎ (14 P)",29.741460,-95.376710
7,"► Neartown, Houston‎ (1 C, 22 P)",29.742719,-95.328188
8,"► Spring Branch, Houston‎ (7 P)",29.803517,-95.515896
9,"► Third Ward, Houston‎ (1 C, 19 P)",29.663583,-95.229110


In [21]:
# save the DataFrame as CSV file
 houston_df.to_csv("kl_df.csv", index=False)

<big> <b> Create a map of Houston, TX with neighborhoods superimposed on top </b> </big>

In [24]:
# get the coordinates of Houston, TX
address = 'Houston, Texas'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Houston, Texas {}, {}.'.format(Latitude, Longitude))

The geograpical coordinate of Houston, Texas 29.7589382, -95.3676974.


In [27]:
# create map of Toronto using latitude and longitude values
map_houston = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(houston_df['Latitude'], houston_df['Longitude'], houston_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_houston)  
    
map_houston

<big> <b> Using Foursquare API to explore the neighborhoods </b> </big>

In [29]:
# define Foursquare Credentials and Version

CLIENT_ID = '0A030PBRMWXGRIX31NLWORJCZ5AKIPXWBHYMR05YYTHHNFZT' # your Foursquare ID
CLIENT_SECRET = 'LLITK3QTE0GXIDQHAQDOOPG23350SQ1ZYZZ2FVMJH3541NLU' # your Foursquare Secret
VERSION = '20180608'
radius = 2000
LIMIT = 100
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Latitude, Longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=0A030PBRMWXGRIX31NLWORJCZ5AKIPXWBHYMR05YYTHHNFZT&client_secret=LLITK3QTE0GXIDQHAQDOOPG23350SQ1ZYZZ2FVMJH3541NLU&ll=29.7589382,-95.3676974&v=20180608&radius=2000&limit=100'

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d127941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4cbf112b00d837047b8a415c,910 Louisiana St,US,Houston,United States,NaN,81,"[910 Louisiana St, Houston, TX 77002, United S...","[{'label': 'display', 'lat': 29.75967067970318...",29.759671,-95.367728,NaN,77002,TX,Baker Botts LLP,v-1581709944,NaN
1,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",NaN,NaN,NaN,NaN,NaN,NaN,False,5b3b978a56ca62001c94580e,910 Louisiana St,US,Houston,United States,NaN,75,"[910 Louisiana St, Houston, TX 77002, United S...","[{'label': 'display', 'lat': 29.75954376558662...",29.759544,-95.367348,NaN,77002,TX,"Nrg Energy, Inc.",v-1581709944,NaN
2,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b993b14f964a520d76b35e3,910 Louisiana St,US,Houston,United States,at McKinney,41,"[910 Louisiana St (at McKinney), Houston, TX 7...","[{'label': 'display', 'lat': 29.75928328897705...",29.759283,-95.367858,NaN,77002,TX,One Shell Plaza,v-1581709944,NaN
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1579928,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/starbucks-9...,False,4b9ff072f964a520224c37e3,3801 Cullen Blvd,US,Houston,United States,NaN,25,"[3801 Cullen Blvd, Houston, TX 77004, United S...","[{'label': 'display', 'lat': 29.75909, 'lng': ...",29.759090,-95.367500,NaN,77004,TX,Starbucks,v-1581709944,NaN
4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",NaN,NaN,NaN,NaN,NaN,NaN,False,527096a111d2393d1fc331aa,920 Louisiana St,US,Houston,United States,Walker St,23,"[920 Louisiana St (Walker St), Houston, TX 770...","[{'label': 'display', 'lat': 29.75911177161870...",29.759112,-95.367565,NaN,77002,TX,The Houston Club,v-1581709944,NaN


In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Baker Botts LLP,Conference Room,910 Louisiana St,US,Houston,United States,NaN,81,"[910 Louisiana St, Houston, TX 77002, United S...","[{'label': 'display', 'lat': 29.75967067970318...",29.759671,-95.367728,NaN,77002,TX,4cbf112b00d837047b8a415c
1,"Nrg Energy, Inc.",Office,910 Louisiana St,US,Houston,United States,NaN,75,"[910 Louisiana St, Houston, TX 77002, United S...","[{'label': 'display', 'lat': 29.75954376558662...",29.759544,-95.367348,NaN,77002,TX,5b3b978a56ca62001c94580e
2,One Shell Plaza,Office,910 Louisiana St,US,Houston,United States,at McKinney,41,"[910 Louisiana St (at McKinney), Houston, TX 7...","[{'label': 'display', 'lat': 29.75928328897705...",29.759283,-95.367858,NaN,77002,TX,4b993b14f964a520d76b35e3
3,Starbucks,Coffee Shop,3801 Cullen Blvd,US,Houston,United States,NaN,25,"[3801 Cullen Blvd, Houston, TX 77004, United S...","[{'label': 'display', 'lat': 29.75909, 'lng': ...",29.759090,-95.367500,NaN,77004,TX,4b9ff072f964a520224c37e3
4,The Houston Club,Restaurant,920 Louisiana St,US,Houston,United States,Walker St,23,"[920 Louisiana St (Walker St), Houston, TX 770...","[{'label': 'display', 'lat': 29.75911177161870...",29.759112,-95.367565,NaN,77002,TX,527096a111d2393d1fc331aa


In [34]:
address = 'Hoston,TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Houston are {}, {}.'.format(Latitude, Longitude))

The geograpical coordinate of Houston are 29.7589382, -95.3676974.


In [48]:
#Create function to know how many venues there are in Houton, TX

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
#List of Neighborhood that have venues in Houston
venues = getNearbyVenues(names=houston_df['Neighborhood'],
                                   latitudes=houston_df['Latitude'],
                                   longitudes=houston_df['Longitude']
                                  )

venues.head()

►  Gated communities in Houston‎ (5 P)
►  Alief, Houston‎ (1 C, 1 P)
►  Downtown Houston‎ (1 C, 44 P)
►  East End, Houston‎ (13 P)
►  Fourth Ward, Houston‎ (7 P)
►  Houston Heights‎ (10 P)
►  Midtown, Houston‎ (14 P)
►  Neartown, Houston‎ (1 C, 22 P)
►  Spring Branch, Houston‎ (7 P)
►  Third Ward, Houston‎ (1 C, 19 P)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Chicken Plus,29.818672,-95.284178,American Restaurant
1,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Corner Store,29.805046,-95.285149,Convenience Store
2,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Family Dollar,29.815663,-95.285027,Discount Store
3,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Chase Bank,29.799212,-95.273584,Bank
4,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Whataburger,29.808083,-95.301394,Burger Joint


In [51]:
print('{} venues were returned by Foursquare.'.format(venues.shape[0]))
venues.head()

700 venues were returned by Foursquare.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Chicken Plus,29.818672,-95.284178,American Restaurant
1,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Corner Store,29.805046,-95.285149,Convenience Store
2,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Family Dollar,29.815663,-95.285027,Discount Store
3,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Chase Bank,29.799212,-95.273584,Bank
4,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Whataburger,29.808083,-95.301394,Burger Joint


In [53]:
#Number of venues per neighborhood
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"► Alief, Houston‎ (1 C, 1 P)",68,68,68,68,68,68
"► Downtown Houston‎ (1 C, 44 P)",100,100,100,100,100,100
"► East End, Houston‎ (13 P)",71,71,71,71,71,71
"► Fourth Ward, Houston‎ (7 P)",100,100,100,100,100,100
► Gated communities in Houston‎ (5 P),14,14,14,14,14,14
► Houston Heights‎ (10 P),100,100,100,100,100,100
"► Midtown, Houston‎ (14 P)",100,100,100,100,100,100
"► Neartown, Houston‎ (1 C, 22 P)",54,54,54,54,54,54
"► Spring Branch, Houston‎ (7 P)",48,48,48,48,48,48


In [54]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 166 uniques categories.


In [55]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
houston_onehot = onehot[fixed_columns]
onehot.head()

,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Poke Place,Polish Restaurant,Portuguese Restaurant,Pub,Public Art,Record Shop,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Travel & Transport,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,► Gated communities in Houston‎ (5 P)
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,► Gated communities in Houston‎ (5 P)
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,► Gated communities in Houston‎ (5 P)
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,► Gated communities in Houston‎ (5 P)
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [56]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Poke Place,Polish Restaurant,Portuguese Restaurant,Pub,Public Art,Record Shop,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Travel & Transport,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"► Alief, Houston‎ (1 C, 1 P)",0.000000,0.000000,0.0,0.00,0.058824,0.00,0.000000,0.000000,0.029412,0.000000,0.014706,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.044118,0.000000,0.0,0.0,0.00,0.014706,0.0,0.000000,0.000000,0.00,0.000000,0.014706,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0,0.014706,0.014706,0.000000,0.029412,0.00,0.00,0.0,0.014706,0.00,0.000000,0.014706,0.029412,0.0,0.00,0.00,0.014706,0.00,0.029412,0.000000,0.014706,0.0,0.073529,0.00,0.00,0.0,0.0,0.014706,0.0,0.00,0.029412,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.014706,0.000000,0.0,0.00,0.014706,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.044118,0.0,0.00,0.0,0.000000,0.029412,0.00,0.00,0.00,0.014706,0.029412,0.0,0.000000,0.000000,0.00,0.0,0.0,0.014706,0.00,0.014706,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.014706,0.014706,0.0,0.014706,0.00,0.029412,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.014706,0.00,0.0,0.00,0.00,0.014706,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.0,0.00
1,"► Downtown Houston‎ (1 C, 44 P)",0.030000,0.000000,0.0,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.000000,0.100000,0.0,0.03,0.01,0.02,0.03,0.01,0.02,0.00,0.0,0.000000,0.03,0.000000,0.010000,0.0,0.0,0.01,0.010000,0.0,0.000000,0.010000,0.04,0.020000,0.000000,0.01,0.0,0.000000,0.000000,0.00,0.00,0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.0,0.000000,0.01,0.000000,0.000000,0.000000,0.0,0.01,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.0,0.000000,0.01,0.00,0.0,0.0,0.000000,0.0,0.00,0.000000,0.01,0.01,0.0,0.01,0.00,0.

In [57]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----►  Alief, Houston‎ (1 C, 1 P)----
                   venue  freq
0  Vietnamese Restaurant  0.25
1            Gas Station  0.07
2       Asian Restaurant  0.06
3        Bubble Tea Shop  0.04
4         Massage Studio  0.04


----►  Downtown Houston‎ (1 C, 44 P)----
                venue  freq
0                 Bar  0.10
1               Hotel  0.09
2  Mexican Restaurant  0.06
3        Cocktail Bar  0.04
4  Italian Restaurant  0.04


----►  East End, Houston‎ (13 P)----
                  venue  freq
0    Mexican Restaurant  0.10
1           Pizza Place  0.07
2        Breakfast Spot  0.06
3     Mobile Phone Shop  0.06
4  Fast Food Restaurant  0.06


----►  Fourth Ward, Houston‎ (7 P)----
         venue  freq
0  Coffee Shop  0.08
1          Bar  0.04
2        Trail  0.04
3         Park  0.04
4          Pub  0.04


----►  Gated communities in Houston‎ (5 P)----
                  venue  freq
0  Fast Food Restaurant  0.21
1           Gas Station  0.14
2   American Restaurant  0.07
3         

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"► Alief, Houston‎ (1 C, 1 P)",Vietnamese Restaurant,Gas Station,Asian Restaurant,Bubble Tea Shop,Massage Studio
1,"► Downtown Houston‎ (1 C, 44 P)",Bar,Hotel,Mexican Restaurant,Cocktail Bar,Italian Restaurant
2,"► East End, Houston‎ (13 P)",Mexican Restaurant,Pizza Place,Fast Food Restaurant,Mobile Phone Shop,Breakfast Spot
3,"► Fourth Ward, Houston‎ (7 P)",Coffee Shop,Park,Pub,Bar,Trail
4,► Gated communities in Houston‎ (5 P),Fast Food Restaurant,Gas Station,American Restaurant,Sandwich Place,Outdoors & Recreation


In [61]:
# set number of clusters
kclusters = 2

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1], dtype=int32)

In [64]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

merged = houston_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged = merged.dropna()


merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,► Gated communities in Houston‎ (5 P),29.806982,-95.283198,Fast Food Restaurant,Gas Station,American Restaurant,Sandwich Place,Outdoors & Recreation
1,"► Alief, Houston‎ (1 C, 1 P)",29.714976,-95.589785,Vietnamese Restaurant,Gas Station,Asian Restaurant,Bubble Tea Shop,Massage Studio
2,"► Downtown Houston‎ (1 C, 44 P)",29.753970,-95.352511,Bar,Hotel,Mexican Restaurant,Cocktail Bar,Italian Restaurant
3,"► East End, Houston‎ (13 P)",29.694616,-95.278793,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Mobile Phone Shop,Breakfast Spot
4,"► Fourth Ward, Houston‎ (7 P)",29.757620,-95.384490,Coffee Shop,Park,Pub,Bar,Trail
